In [31]:
import os
import matplotlib.pyplot as plt
from tqdm import tqdm
import shutil
import PIL
import pandas as pd
from libtiff import TIFF
import numpy as np
import re
from tifffile import tifffile

from sklearn.model_selection import StratifiedShuffleSplit, train_test_split
from keras.preprocessing.image import ImageDataGenerator
import pandas as pd
import tensorflow as tf
from keras.models import Model
from keras.layers import Dense, Dropout, Flatten
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras.optimizers import Adam
from keras.applications import ResNet50
from keras.utils import to_categorical


In [107]:
import os
import numpy as np
from tqdm import tqdm
from tifffile import imread, imwrite

DATA_DIR= "/home/fatima_tuz_zehra/Dataset/eurosat-all"
model_dir = "/home/fatima_tuz_zehra/Dataset/euro-rgb-new"
ms_dir = "/home/fatima_tuz_zehra/Dataset/euro-all-assets"
data_path = "/home/fatima_tuz_zehra/Dataset"

#ds_sizes = {label: len(os.listdir(os.path.join(DATA_DIR,label))) for label in classes}

if not os.path.isdir(os.path.join(data_path,'ms_and_ind')):
  os.mkdir(os.path.join(data_path,'ms_and_ind'))
for land_class in classes:
  class_path = os.path.join(os.path.join(data_path,'ms_and_ind'), land_class)
  if not os.path.isdir(class_path):
      os.mkdir(class_path)





In [108]:
reject = [0, 11, 12]
accept = [i for i in range(13) if i not in reject]

for land_class in tqdm(classes):
  class_path = os.path.join(DATA_DIR, land_class)
  image_fps = [os.path.join(class_path, img_name) for img_name in os.listdir(class_path)]
  for path in tqdm(image_fps):
   img = imread(path)
   req_bands = img[accept]
   # indices = get_indices(img)
   #result = np.vstack((req_bands, indices))
   res_path = os.path.join('ms_and_ind', land_class, path.split('/')[-1])
   imwrite(res_path, req_bands)

100%|██████████| 10/10 [02:12<00:00, 13.24s/it]


In [109]:
print(tf.test.is_gpu_available())

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
True


In [110]:
path = os.path.join(data_path, "ms_and_ind")
#path = "/home/fatima_tuz_zehra/Dataset/eurosat-all"

labels = os.listdir(path)
labels

['HerbaceousVegetation',
 'Highway',
 'SeaLake',
 'Forest',
 'River',
 'Pasture',
 'PermanentCrop',
 'Industrial',
 'Residential',
 'AnnualCrop']

In [112]:
data = dict()
for label in labels:
    imgs = os.listdir(os.path.join(DATA_DIR,label))
    for img in imgs:
        data[os.path.join(DATA_DIR,label,img)] = label
        
data = pd.DataFrame(data.items())
y = data[1]
X_train, X_test, y_train, y_test = train_test_split(data, y, test_size=0.2, random_state=42,shuffle=True)

In [113]:
X_train.to_csv(os.path.join(ms_dir,"train.csv"),index = False)
X_test.to_csv(os.path.join(ms_dir,"test.csv"),index = False)

In [114]:
tr = pd.read_csv(os.path.join(ms_dir,"train.csv"))
ts = pd.read_csv(os.path.join(ms_dir,"test.csv"))

In [115]:
tr[0:6]['0']

0    /home/fatima_tuz_zehra/Dataset/eurosat-all/Pas...
1    /home/fatima_tuz_zehra/Dataset/eurosat-all/Ind...
2    /home/fatima_tuz_zehra/Dataset/eurosat-all/Her...
3    /home/fatima_tuz_zehra/Dataset/eurosat-all/Sea...
4    /home/fatima_tuz_zehra/Dataset/eurosat-all/Ind...
5    /home/fatima_tuz_zehra/Dataset/eurosat-all/Sea...
Name: 0, dtype: object

In [116]:
for each in tr[0:6]['0']:
    print(each)

/home/fatima_tuz_zehra/Dataset/eurosat-all/Pasture/Pasture_1456.tif
/home/fatima_tuz_zehra/Dataset/eurosat-all/Industrial/Industrial_173.tif
/home/fatima_tuz_zehra/Dataset/eurosat-all/HerbaceousVegetation/HerbaceousVegetation_2696.tif
/home/fatima_tuz_zehra/Dataset/eurosat-all/SeaLake/SeaLake_2051.tif
/home/fatima_tuz_zehra/Dataset/eurosat-all/Industrial/Industrial_1148.tif
/home/fatima_tuz_zehra/Dataset/eurosat-all/SeaLake/SeaLake_287.tif


In [118]:
def data_generator(file = "train", batch_size = 32):
    
    batch_Y= []
    idx = 0
    
    if file == "train":
        df = pd.read_csv(os.path.join(ms_dir,"train.csv"))
        y = pd.get_dummies(df['1'])
    else:
        df = pd.read_csv(os.path.join(ms_dir,"test.csv"))
        y = pd.get_dummies(df['1'])
        
    num_records = len(df)
    i = 0
    
    while i != -1:
        batch_X = []
        
        idx += batch_size
        if idx < len(df)-1:
            samples = df[idx-batch_size:idx]['0']
            batch_Y = y[idx-batch_size:idx]
            
        else:
            i = -1
            samples = df[idx-batch_size:len(df)]['0']
            batch_Y = y[idx-batch_size:len(df)]
            
            
       
        
        for sample in samples:
            image = tifffile.imread(sample)
            x = np.moveaxis(image,0,-1)
            batch_X.append(x)
        
        
    
        yield [np.array(batch_X)], batch_Y.to_numpy()
    
    return
            

                           
            
        
        
    

In [119]:
def get_model(lr, tune = None):
    resnet = ResNet50(include_top=False,
                                 weights=None ,
                                 input_shape=(64,64,10))
    model = resnet.output
    model = Flatten()(model)
    model = Dense(2048, activation='relu')(model)
    model = Dropout(0.3)(model)

    output = Dense(10, activation='softmax')(model)

    model = Model(inputs=resnet.input, outputs=output)
    
    if(tune == 0):
        for layer in resnet.layers:
            layer.trainable = True
            
    else:
        for layer in resnet.layers:
            layer.trainable = False
        

    model.compile(optimizer=Adam(lr = lr), loss='categorical_crossentropy',
                     metrics=['categorical_accuracy'])
    return model

In [120]:
rgb_model = tf.keras.models.load_model(os.path.join(model_dir,"best.hdf5"))

In [69]:
model = get_model(lr = 1e-3,tune=0)

In [70]:
for layer in rgb_model.layers:
    print(layer)
    weights = layer.get_weights()
    for i in range(len(weights)):
        print(weights[i].shape)

(7, 7, 3, 64)
(64,)
(64,)
(64,)
(64,)
(64,)
(1, 1, 64, 64)
(64,)
(64,)
(64,)
(64,)
(64,)
(3, 3, 64, 64)
(64,)
(64,)
(64,)
(64,)
(64,)
(1, 1, 64, 256)
(256,)
(1, 1, 64, 256)
(256,)
(256,)
(256,)
(256,)
(256,)
(256,)
(256,)
(256,)
(256,)
(1, 1, 256, 64)
(64,)
(64,)
(64,)
(64,)
(64,)
(3, 3, 64, 64)
(64,)
(64,)
(64,)
(64,)
(64,)
(1, 1, 64, 256)
(256,)
(256,)
(256,)
(256,)
(256,)
(1, 1, 256, 64)
(64,)
(64,)
(64,)
(64,)
(64,)
(3, 3, 64, 64)
(64,)
(64,)
(64,)
(64,)
(64,)
(1, 1, 64, 256)
(256,)
(256,)
(256,)
(256,)
(256,)
(1, 1, 256, 128)
(128,)
(128,)
(128,)
(128,)
(128,)
(3, 3, 128, 128)
(128,)
(128,)
(128,)
(128,)
(128,)
(1, 1, 256, 512)
(512,)
(1, 1, 128, 512)
(512,)
(512,)
(512,)
(512,)
(512,)
(512,)
(512,)
(512,)
(512,)
(1, 1, 512, 128)
(128,)
(128,)
(128,)
(128,)
(128,)
(3, 3, 128, 128)
(128,)
(128,)
(128,)
(128,)
(128,)
(1, 1, 128, 512)
(512,)
(512,)
(512,)
(512,)
(512,)
(1, 1, 512, 128)
(128,)
(128,)
(128,)
(128,)
(128,)
(3, 3, 128, 128)
(128,)
(128,)
(128,)
(128,)
(128,)
(1, 1, 128, 

In [71]:
for layer in model.layers:
    print(layer)
    weights = layer.get_weights()
    for i in range(len(weights)):
        print(weights[i].shape)

(7, 7, 10, 64)
(64,)
(64,)
(64,)
(64,)
(64,)
(1, 1, 64, 64)
(64,)
(64,)
(64,)
(64,)
(64,)
(3, 3, 64, 64)
(64,)
(64,)
(64,)
(64,)
(64,)
(1, 1, 64, 256)
(256,)
(1, 1, 64, 256)
(256,)
(256,)
(256,)
(256,)
(256,)
(256,)
(256,)
(256,)
(256,)
(1, 1, 256, 64)
(64,)
(64,)
(64,)
(64,)
(64,)
(3, 3, 64, 64)
(64,)
(64,)
(64,)
(64,)
(64,)
(1, 1, 64, 256)
(256,)
(256,)
(256,)
(256,)
(256,)
(1, 1, 256, 64)
(64,)
(64,)
(64,)
(64,)
(64,)
(3, 3, 64, 64)
(64,)
(64,)
(64,)
(64,)
(64,)
(1, 1, 64, 256)
(256,)
(256,)
(256,)
(256,)
(256,)
(1, 1, 256, 128)
(128,)
(128,)
(128,)
(128,)
(128,)
(3, 3, 128, 128)
(128,)
(128,)
(128,)
(128,)
(128,)
(1, 1, 256, 512)
(512,)
(1, 1, 128, 512)
(512,)
(512,)
(512,)
(512,)
(512,)
(512,)
(512,)
(512,)
(512,)
(1, 1, 512, 128)
(128,)
(128,)
(128,)
(128,)
(128,)
(3, 3, 128, 128)
(128,)
(128,)
(128,)
(128,)
(128,)
(1, 1, 128, 512)
(512,)
(512,)
(512,)
(512,)
(512,)
(1, 1, 512, 128)
(128,)
(128,)
(128,)
(128,)
(128,)
(3, 3, 128, 128)
(128,)
(128,)
(128,)
(128,)
(128,)
(1, 1, 128,

In [85]:
channel_weights = rgb_model.layers[2].get_weights()
diff_w = channel_weights[0]

multi_channel_weights = model.layers[2].get_weights()
rep_w = multi_channel_weights[0]

for i in range(10):
    if i < 3 :
        rep_w[:,:,i,:] = diff_w[:,:,i,:]
    else:
        rep_w[:,:,i,:] = diff_w[:,:,0,:]
rep_single = rgb_model.layers[2].get_weights()[1]
arr_weights = [rep_w, rep_single]

for i in range(len(model.layers)):
    if i == 2:
        model.layers[i].set_weights(arr_weights)
    else:
        model.layers[i].set_weights(rgb_model.layers[i].get_weights())

model.save(os.path.join(model_dir,"loaded-ms.hdf5"))        

In [86]:
pretrained = tf.keras.models.load_model(os.path.join(model_dir,"loaded-ms.hdf5"))

In [101]:
checkpoint = ModelCheckpoint(filepath =  ms_dir + '/all-bands.hdf5',
                        monitor='val_categorical_accuracy',
                        save_best_only=True,
                        verbose=1)

for i in range(10):
    train_gen = data_generator(file = "train", batch_size = 32)
    test_gen = data_generator(file = "test", batch_size = 16)
    pretrained.fit(train_gen, steps_per_epoch=len(tr)//32, epochs=1, validation_data=test_gen,validation_steps=len(ts)//16,callbacks=[checkpoint])

675/675 [==============================] - ETA: 0s - loss: 1.0770 - categorical_accuracy: 0.7908
Epoch 00001: val_categorical_accuracy improved from -inf to 0.74017, saving model to /home/fatima_tuz_zehra/Dataset/euro-rgb-new/all-bands.hdf5
675/675 [==============================] - ETA: 0s - loss: 0.4324 - categorical_accuracy: 0.8858
Epoch 00001: val_categorical_accuracy improved from 0.74017 to 0.84996, saving model to /home/fatima_tuz_zehra/Dataset/euro-rgb-new/all-bands.hdf5
675/675 [==============================] - ETA: 0s - loss: 0.3554 - categorical_accuracy: 0.9104
Epoch 00001: val_categorical_accuracy did not improve from 0.84996
675/675 [==============================] - ETA: 0s - loss: 0.2772 - categorical_accuracy: 0.9269
Epoch 00001: val_categorical_accuracy did not improve from 0.84996
675/675 [==============================] - ETA: 0s - loss: 0.2934 - categorical_accuracy: 0.9234
Epoch 00001: val_categorical_accuracy did not improve from 0.84996
675/675 [==============

In [102]:
for i in range(3):
    train_gen = data_generator(file = "train", batch_size = 32)
    test_gen = data_generator(file = "test", batch_size = 16)
    pretrained.fit(train_gen, steps_per_epoch=len(tr)//32, epochs=1, validation_data=test_gen,validation_steps=len(ts)//16,callbacks=[checkpoint])

675/675 [==============================] - ETA: 0s - loss: 0.2340 - categorical_accuracy: 0.9234
Epoch 00001: val_categorical_accuracy did not improve from 0.92044
675/675 [==============================] - ETA: 0s - loss: 0.2107 - categorical_accuracy: 0.9314
Epoch 00001: val_categorical_accuracy did not improve from 0.92044
675/675 [==============================] - ETA: 0s - loss: 0.2052 - categorical_accuracy: 0.9332
Epoch 00001: val_categorical_accuracy did not improve from 0.92044
675/675 [==============================] - 68s 101ms/step - loss: 0.2052 - categorical_accuracy: 0.9332 - val_loss: 0.9372 - val_categorical_accuracy: 0.7342


In [103]:
pretrained.load_weights(ms_dir + '/all-bands.hdf5')
pretrained.optimizer.lr = 0.0001
for i in range(10):
    train_gen = data_generator(file = "train", batch_size = 32)
    test_gen = data_generator(file = "test", batch_size = 16)
    pretrained.fit(train_gen, steps_per_epoch=len(tr)//32, epochs=1, validation_data=test_gen,validation_steps=len(ts)//16,callbacks=[checkpoint])

675/675 [==============================] - ETA: 0s - loss: 0.1589 - categorical_accuracy: 0.9488
Epoch 00001: val_categorical_accuracy improved from 0.92044 to 0.94881, saving model to /home/fatima_tuz_zehra/Dataset/euro-rgb-new/all-bands.hdf5
675/675 [==============================] - ETA: 0s - loss: 0.1252 - categorical_accuracy: 0.9601
Epoch 00001: val_categorical_accuracy did not improve from 0.94881
675/675 [==============================] - ETA: 0s - loss: 0.1045 - categorical_accuracy: 0.9658
Epoch 00001: val_categorical_accuracy did not improve from 0.94881
675/675 [==============================] - ETA: 0s - loss: 0.0874 - categorical_accuracy: 0.9718
Epoch 00001: val_categorical_accuracy improved from 0.94881 to 0.95215, saving model to /home/fatima_tuz_zehra/Dataset/euro-rgb-new/all-bands.hdf5
675/675 [==============================] - ETA: 0s - loss: 0.0709 - categorical_accuracy: 0.9757
Epoch 00001: val_categorical_accuracy did not improve from 0.95215
675/675 [===========